In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load the training dataset
train_csv_path = './archive/sign_mnist_train.csv'
train_data = pd.read_csv(train_csv_path)

# Load the testing dataset
test_csv_path = './archive/sign_mnist_test.csv'
test_data = pd.read_csv(test_csv_path)

In [3]:
# Preprocess the data
X_train = train_data.drop('label', axis=1).values
y_train = train_data['label'].values

X_test = test_data.drop('label', axis=1).values
y_test = test_data['label'].values

# Normalize pixel values
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [4]:
class CustomSVM:
    def __init__(self, C=1, kernel='linear', degree=3, gamma='auto'):
        """
        Initialize the Custom SVM Classifier.
        
        Parameters:
        - C (float): Penalty parameter of the error term.
        - kernel (str): Kernel function to use ('linear', 'poly', 'rbf').
        - degree (int): Degree of the polynomial kernel.
        - gamma (str or float): Kernel coefficient for 'rbf' and 'poly' kernels.
        """
        self.C = C
        self.kernel = kernel
        self.degree = degree
        self.gamma = gamma
        self.alpha = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.intercept = None

    def _kernel(self, X1, X2):
        """
        Compute the kernel function between two sets of samples.
        
        Parameters:
        - X1 (numpy.ndarray): First set of samples.
        - X2 (numpy.ndarray): Second set of samples.
        
        Returns:
        - Kernel matrix.
        """
        if self.kernel == 'linear':
            return np.dot(X1, X2.T)
        elif self.kernel == 'poly':
            return (1 + np.dot(X1, X2.T)) ** self.degree
        elif self.kernel == 'rbf':
            if self.gamma == 'auto':
                self.gamma = 1 / X1.shape[1]
            return np.exp(-self.gamma * np.sum((X1[:, None] - X2) ** 2, axis=-1))

    def fit(self, X, y):
        """
        Train the Custom SVM Classifier.
        
        Parameters:
        - X (numpy.ndarray): Training samples.
        - y (numpy.ndarray): Training labels.
        """
        n_samples, n_features = X.shape
        
        # Initialize Lagrange multipliers
        self.alpha = np.zeros(n_samples)
        
        # Compute kernel matrix
        K = self._kernel(X, X)
        
        # Optimize Lagrange multipliers
        for _ in range(1000):
            for i in range(n_samples):
                grad = np.sum(self.alpha * y * K[:, i]) - 1
                if (y[i] * grad < -1e-5) and (self.alpha[i] < self.C):
                    self.alpha[i] += y[i]
        
        # Identify support vectors
        self.support_vectors = X[self.alpha > 1e-5]
        self.support_vector_labels = y[self.alpha > 1e-5]
        
        # Compute intercept
        self.intercept = np.mean(self.support_vector_labels -
                                np.dot(self._kernel(self.support_vectors, self.support_vectors),
                                       self.support_vector_labels * self.alpha[self.alpha > 1e-5]))

    def predict(self, X):
        """
        Make predictions using the trained Custom SVM Classifier.
        
        Parameters:
        - X (numpy.ndarray): Samples to predict.
        
        Returns:
        - Predicted labels.
        """
        decision_values = np.dot(self._kernel(X, self.support_vectors),
                                self.support_vector_labels * self.alpha[self.alpha > 1e-5]) + self.intercept
        return np.sign(decision_values)

In [ ]:
# Example usage
svm = CustomSVM(C=1, kernel='rbf', gamma='auto')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)